# 랜덤 포레스트 훈련
- 랜덤 포레스트(Random Forest)는 앙상블 학습의 대표. 결정 트리를 랜덤하게 만들어 결정 트리(나무)의 숲을 만듦. 그리고 각 결정 트리의 예측을 사용해 최종 예측을 만듦

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
wine = pd.read_csv('https://bit.ly/wine_csv_data')
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [2]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

In [3]:
rf = RandomForestClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


In [6]:
rf.fit(train_input, train_target)
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


In [8]:
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)
rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8934000384837406


# 엑스트라 트리
- 엑스트라 트리(Extra Trees)는 랜덤 포레스트와 매우 비슷. 기본적으로 100개의 결정트리 훈련. 랜덤 포레스트와 동일하게 결정 트리가 제공하는 대부분의 매개변수를 지원. 전체 특성 중 일부 특성을 랜덤하게 선택하여 노드를 분할하는데 사용.
- 랜덤 포레스트와 엑스트라 트리의 차이점은 부트스트랩 샘플을 사용하지 않는다는 점. 즉각 결정 트리를 만들 때 전체 훈련 세트를 사용. 노드를 분할할 때 가장 좋은 분할을 찾는 것이 아니라 무작위로 분할.

In [9]:
from sklearn.ensemble import ExtraTreesClassifier

In [10]:
et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)

In [11]:
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


In [12]:
et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


# 그레이디언트 부스팅
- 그레이디언트 부스팅(gradient boosting)은 깊이가 얕은 결정 트리를 사용하여 이전 트리의 오차를 보완하는 방식으로 앙상블.
- 사이킷런의 GradientBoostingClassifier는 기본적으로 깊이가 3인 결정 트리를 100개 사용. 깊이가 얕은 결정 트리를 사용하기 때문에 과대적합에 강하고 일반적으로 높은 일반화 성능을 기대할 수 있습니다.

In [13]:
from sklearn.ensemble import GradientBoostingClassifier

In [14]:
gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


In [15]:
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


In [16]:
gb.fit(train_input, train_target)
print(gb.feature_importances_)

[0.15872278 0.68011572 0.16116151]


# 히스토그램 기반 그레이디언트 부스팅
- 히스토그램 기반 그레이디언트 부스팅(Histogram-based Gradient Boosting)은 정형 데이터를 다루는 머신러닝 알고리즘 중에 가장 인기가 높은 알고리즘.
- 히스토그램 기반 ㅓ그레이디언트 부스팅은 256개의 구간 중에서 하나를 떼어 놓고 누락된 값을 위해서 사용. 따라서 전처리 필요 X

In [17]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

c:\Users\user\anaconda3\Lib\site-packages\sklearn\experimental\enable_hist_gradient_boosting.py:15: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  warnings.warn(


In [20]:
hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9321723946453317 0.8801241948619236


# Permutation Importance(치환 중요도)
- 특성을 하나씩 랜덤하게 섞어서 모델의 성능이 변화하는지를 관찰하여 어떤 특성이 중요한지를 계산.
- 훈련 세트뿐만 아니라 테스트 세트에도 적용하라 수 있고 사이킷런에서 제공하는 추정기 모델에 모두 사용할 수 있다.

In [23]:
from sklearn.inspection import permutation_importance
hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target, n_repeats=10, random_state=42, n_jobs=-1)
print(result.importances_mean)


[0.08876275 0.23438522 0.08027708]


In [24]:
result = permutation_importance(hgb, test_input, test_target, n_repeats=10, random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.05969231 0.20238462 0.049     ]


In [25]:
hgb.score(test_input, test_target)

0.8723076923076923

# XGBoost vs LightGBM

In [28]:
pip install xgboost

   ---------------------------------------- 0.0/99.8 MB ? eta -:--:--
   ---------------------------------------- 0.1/99.8 MB 1.7 MB/s eta 0:01:01
   ---------------------------------------- 0.2/99.8 MB 2.8 MB/s eta 0:00:37
   ---------------------------------------- 0.6/99.8 MB 4.6 MB/s eta 0:00:22
   ---------------------------------------- 0.9/99.8 MB 5.4 MB/s eta 0:00:19
    --------------------------------------- 1.7/99.8 MB 8.5 MB/s eta 0:00:12
   - -------------------------------------- 3.5/99.8 MB 13.8 MB/s eta 0:00:07
   --- ------------------------------------ 7.9/99.8 MB 26.4 MB/s eta 0:00:04
   --- ------------------------------------ 8.7/99.8 MB 29.3 MB/s eta 0:00:04
   ----- ---------------------------------- 12.6/99.8 MB 59.8 MB/s eta 0:00:02
   ------ --------------------------------- 15.7/99.8 MB 59.5 MB/s eta 0:00:02
   -------- ------------------------------- 21.3/99.8 MB 81.8 MB/s eta 0:00:01
   --------- ------------------------------ 23.7/99.8 MB 110.0 MB/s eta 0:


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [29]:
from xgboost import XGBClassifier

In [30]:
xgb = XGBClassifier(tree_method = 'hist', random_state=42)
scores = cross_validate(xgb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9558403027491312 0.8782000074035686


In [31]:
from lightgbm import LGBMClassifier

In [32]:
lgb = LGBMClassifier(random_state=42)
scores = cross_validate(lgb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.935828414851749 0.8801251203079884


# 앙상블 보고서
- 앙상블 학습은 정형 데이터에서 가장 뛰어난 성능을 내는 머신러닝 알고리즘 중 하나.

## 사이킷런
- 랜덤 포레스트: 부트스트랩 샘플 사용. 대표 앙상블 학습 알고리즘.
- 엑스트라 트리: 결정 트리의 노드를 랜덤하게 분할함
- 그레이디언트 부스팅: 이전 트리의 손실을 보완하는 식으로 얕은 결정트리를 연속하여 추가.
- 히스토그램 기반 그레이디언트 부스팅: 훈련 데이터를 256개 정수 구간으로 나누어 빠르고 높은 성능을 냄.
## 그외 라이브러리
- XGBoost
- LightGBM